In [20]:
from ris import db2
import datetime
import re
import configparser
#from helper import *
from IPython.display import clear_output
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

config = configparser.ConfigParser()
config.read('db.cfg')

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%m-%d-%Y')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-06-15_11:43
E:\RIS\Staff Folders\Samuel\Requests\SH\RIS DB


In [21]:
cdb = db2.PostgresDb('DOTDEVRHPGSQL01', 'CRASHDATA', quiet = True)
gdb = db2.SqlDb('dotgissql01', 'gisgrid', user='GISUSER', db_pass='GISUSER') #Database Connection

User name (CRASHDATA):soge
Password (CRASHDATA)········


In [30]:
def xy_to_geom(cdb,*args): 
    
    """
    Function that generates the hexidecimal geomteric string 
    of every input coordinate. 
   
    Args:
        cdb param: Database Connection Function for PostgreSQL 
        
    *args
        xy arg: X,Y coordinates 
                format: [[x,y][x,y]] or xy=[x,y] or input x=x,y=y

    Returns:
        geom (str): Hexidecimal geomteric string 
        
    
    e.g.
    
    Input:
    
    xy_to_geom(cdb,[[1000245.0, 202806.0],[1000537.9999999995, 186726.9999999482],[1002334.9999999987, 250597.99999993874]])
    
    Returns:
    
    ['0101000020D7080000000000006A862E4100000000B0C10841',
     '0101000020D708000000000000B4882E410000000038CB0641']
    """
    
   
    
    try:
        
        if any(isinstance(item, list) for item in args[0]):  #args is a tuple 
            #print('TRUE')
            count=0
            geom =[None]*len(args[0])
            for i in args[0]:
                g=cdb.query("select ST_GeomFROMText('POINT(' || {x}::text || ' ' ||  {y}::text || ')', 2263) geom ".format(x=i[0],y=i[1]))
                #print(g)
                geom[count]=g[0][0][0]
                count+=1

        else:
            #print(args[0])
            if isinstance(args[0], list):
                g = cdb.query("select ST_GeomFROMText('POINT(' || {x}::text || ' ' ||  {y}::text || ')', 2263) geom".format(x=args[0][0],y=args[0][1]))
                geom = g[0][0][0]

    except:
    
        xy = list(args)
        #print(lon_lat)
        g = cdb.query("select ST_GeomFROMText('POINT(' || {x}::text || ' ' ||  {y}::text || ')', 2263) geom".format(x=xy[0],y=xy[1]))
        geom = g[0][0][0]

    return geom

In [33]:
def lon_lat_to_geom(cdb,*args): #input either list of list [[lon,lat][lon,lat]] or list lon_lat=[lon,lat] or input lon=lon,lat=lat 
    
    """
    Function that generates the hexidecimal geomteric string 
    of every input coordinate. 
   
    Args:
        cdb param: Database Connection Function for PostgreSQL 
        
    *args
        lon_lat arg (float): Longitude,Latitude coordinates 
                             format: [[lon,lat][lon,lat]] or [lon,lat] or input lon=lon,lat=lat

    Returns:
        geom (str): Hexidecimal geomteric string 
        
    
    e.g.
    
    Input:
    
    lon_lat_to_geom(cdb,[[-73.99247600,40.71507200],[-73.99247600,40.71507200]])
    
    Returns:
    
    ['0101000020D7080000D5011E94BF192E41347ACDE4A4630841',
     '0101000020D7080000D5011E94BF192E41347ACDE4A4630841']
    """
    
    
  
   
    try:
        if any(isinstance(item, list) for item in args[0]): 
            print(args)
            #print('TRUE')
            count=0
            geom =[None]*len(args[0])
            for i in args[0]:
                #print(i)
                g= cdb.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=i[0],lat=i[1]))
                #print(g)
                geom[count]=g[0][0][0]
                count+=1

        else:
            print(args)
            if isinstance(args[0], list):
                g = cdb.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=args[0][0],lat=args[0][1]))
                geom = g[0][0][0]

    except:
        lon_lat = list(args)
        print(lon_lat)
        g = cdb.query("select st_transform(ST_PointFromText('POINT(' || {lon} || ' ' || {lat} || ')', 4326), 2263) geom ".format(lon=lon_lat[0],lat=lon_lat[1]))
        geom = g[0][0][0]

    return geom

In [10]:
def to_itx(cdb,**args): 
    
    """
    Function that retrieves nearest intersection from the location
    of input control signal
   
    Args:
        cdb param: Database Connection Function for PostgreSQL
        
    *args:
        rad (int, optional): Search radius for locating nearest node from all way stop
        xy arg (optional if geom given): X,Y coordinates of all way stop
                                         format: [[x,y][x,y]] or xy=[x,y] or input x=x,y=y
        lon_lat arg (optional if geom given): Longitude,Latitude coordinates 
                                              format: [[lon,lat][lon,lat]] or [lon,lat] or input lon=lon, lat=lat
        geom arg (str, optional if xy or lon_lat given): Hexidecimal geomteric string X,Y coordinates of all way stop
        
    Returns:
        result (dataFrame): Nodeid, Distance between Nodeid and Control Signal, Nodeid Geometry, Control geometry, Coordinates
        val (int): 0 if no args given`    
        
    e.g.
    
    Input:
    
    to_itx(cdb,xy=[1000537.9999999995, 186726.9999999482],geom='0101000020D708000000000000B4882E410000000038CB0641')
    
    Returns:
    
    nodeid  distance    node_geom                   control_geom                xy
    26723   0.0340567   0101000020D7080000009...    0101000020D7080000000...    [1000538.0, 186727.0]
    """  
    
    db= cdb
    rad = args.get('rad', 50)  
    geom = args.get('geom', 0)
    xy = args.get('xy', 0)
    lon_lat= args.get('lon_lat', 0)
    coor = lambda x: map(float,x.replace('[','').replace(']','').split(','))  
    
         
    if len(args)==0:
            return 0
    
    if geom==0:
        if xy:
            geom = xy_to_geom(db,xy)
        else:
            geom = lon_lat_to_geom(db,lon_lat)

    
    if xy:
        
        query = """select nodeid,  masterid, st_distance(geom, '{geom}'::geometry) as distance,
                            geom as node_geom, '{geom}'::geometry as control_geom,
                            '{xy}' as xy,  ST_AsText(st_transform(geom::text,4326)) node_lon_lat,
                            ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
               from node where st_dwithin('{geom}'::geometry,geom, {rad})
                        and is_int = true
               order by st_distance(geom, '{geom}'::geometry) ASC limit 1
               """.format(geom=geom, rad=rad, xy=xy)
        
        r = db.query(query)[0][0]
        x=list(r[0:7])
        x.append(coor(r[4]))
        
        data=x
        #data = pd.DataFrame(x,index=['nodeid','distance','node_geom','control_geom','xy']).T
        
    elif lon_lat:

        query = """
                   select *, ST_AsText(st_transform(node_geom::text,4326)) node_lon_lat, ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from(
                   select nodeid,  masterid, st_distance(geom, '{geom}'::geometry) as distance,
                                geom as node_geom,
                                '{geom}'::geometry as sig_geom  
                   from node where st_dwithin('{geom}'::geometry,geom, {rad})
                            and is_int = true 
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1) x 
                   """.format(geom=geom, rad=rad, lon_lat=lon_lat)

        r = db.query(query)[0][0]
        x=list(r[0:7])
        #x.append(coor(r[4]))

        data=x
        #data = pd.DataFrame(x,index=['nodeid','distance','node_geom','control_geom','lon_lat']).T     
            
    else:
        
        query = """select nodeid, masterid, st_distance(geom, '{geom}'::geometry) as distance,
                                geom as node_geom, '{geom}'::geometry as control_geom,
                                ST_AsText(st_transform(geom::text,4326)) node_lon_lat,
                                ST_AsText(st_transform('{geom}'::text,4326)) control_lon_lat
                   from node where st_dwithin('{geom}'::geometry,geom, {rad})
                            and is_int = true
                   order by st_distance(geom, '{geom}'::geometry) ASC limit 1
                   """.format(geom=geom, rad=rad)
            
        r = list(db.query(query)[0][0])
        data = pd.DataFrame(r,index=['nodeid','distance','node_geom','control_geom', 'node_lon_lat', 'control_lon_lat']).T
    


    return data

In [11]:
def all_sigs(gdb,cdb,rad=50):
    
    
    """
    Function that creates dataframe of all signal longitude & latitude coordinates with nearest intersections and details

    
    Args:
        gdb param: Database Connection Function for GISGRID Database
        cdb param: Database Connection Function for PostgreSQL
        rad (int,optional): Search radius for locating nearest node from signal. Defaults to rad = 300.
        
    Returns:
        sig_data (List of 2 DataFrames): sig_data[0] - Control Signals with node matches within search radius
                                                     - Nodeid, Distance between Nodeid and Signal, Nodeid Geometry, 
                                                       Signal geometry, Signal Longitude & Latitude
                                                       
                                         sig_data[1] - Control Signals that don't have nodes within search radius
                                                     - control_geom,  xy
    
    e.g.
    
    Input:
    
    sig_data = all_sigs(gdb,cdb,50)
    
    Returns:
    
    sig_data[0] - 
    
    nodeid   distance   node_geom                                 control_geom                                  lon_lat
    92892    5.06653    0101000020D708000000853B78B7192E4100...   0101000020D7080000D5011E94BF192E41347AC3...   [-73.992476, 40.715072]
    20243    15.4098    0101000020D708000080BECDD3F5FD2D4100...   0101000020D7080000F253F6F0E1FD2D4169633F...   [-74.005343, 40.721694]
    20216    27.9408    0101000020D708000000FB62772CFE2D4100...   0101000020D708000083DE03842AFE2D41A4B226...   [-74.005212, 40.720552]
    20215    0.545153   0101000020D708000080B210CF38FE2D4100...   0101000020D7080000E5AD4DCA37FE2D412DEDC0...   [-74.005188, 40.719838]
    .....    ........     ..........................................   .......................................  .....................
    
    control_geom                                        lon_lat
    0101000020D708000017E7AB2F7B302E41B94928811651...   [-73.98198, 40.713441]
    0101000020D70800008EBC932D40402E417BE2B7749798...   [-73.974688, 40.742205]
    0101000020D70800002A42A900A03C2E41F58695D20751...   [-73.976365, 40.73592]
    .................................................   .....................
    """
    
    
    """
    Function that creates a list of all signal locations in 
    the SIGNAL_CONTROLLER table of the Gisgrid database.
    
    Args:
        gdb param: Database Connection Function for GISGRID Database
        
    Returns:
        sig_data (list): List containing latitudes and longitues for all sgignals retrieved.
    
    e.g.
    
    Returns [[lat, lon]]:
    
    [[-73.992476, 40.715072],
     [-74.005343, 40.721694],
     [-74.005212, 40.720552],
     .......................]]  
    """    
    
   
    result = gdb.query("""

                    SELECT 'SIG' typ, [PSGM_ID], [Longitude] lon, [Latitude] lat
                    FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                    where NormalizedType != 'Z'
                    and [Longitude] is not null
                    
                        """)
    
  
    sig_data = pd.DataFrame()
    failed_data = pd.DataFrame()

    for i in result.data:

        try:
            s1 = [i[0],i[1]]
            s2 = to_itx(cdb,rad=50,lon_lat=[float(i[2]),float(i[3])])        
            s1.extend(s2)
            sig_data = sig_data.append(pd.DataFrame(s1,['control_type','control_id','nodeid','masterid','distance','node_geom',
                                                        'control_geom','node_lon_lat','control_lon_lat']).T, ignore_index = True)
        except:
            f = [i[0],i[1]]
            f1=lon_lat_to_geom(cdb,[float(i[2]),float(i[3])])
            f.extend([f1])
            failed_data=failed_data.append(pd.DataFrame(f,index = ['control_type','control_id','contol_geom']).T,ignore_index=True)
    
    sig_data['node_lon_lat'] = map(lambda x: map(float,x.strip('POINT()').split(' ')), sig_data['node_lon_lat'])
    sig_data['control_lon_lat'] = map(lambda x: map(float,x.strip('POINT()').split(' ')), sig_data['control_lon_lat'])
    sig_data = sig_data.reindex(columns=['nodeid', 'masterid','control_type', 'control_id', 'node_lon_lat', 'control_lon_lat', 'distance'])
    return [sig_data, failed_data]


In [5]:
   
result = gdb.query("""

                SELECT 'SIG' typ, [PSGM_ID], [Longitude] lon, [Latitude] lat
                FROM [GISGRID].[gisadmin].[SIGNAL_CONTROLLER]
                where NormalizedType != 'Z'
                and [Longitude] is not null

                    """)

	Missing SQL Server Native Client 10.0 datetime2 will not be interpreted correctly



In [12]:
sig_data = pd.DataFrame()
failed_data = pd.DataFrame()

for i in result.data:
    print(i)
    try:
        s1 = [i[0],i[1]]
        s2 = to_itx(cdb,rad=50,lon_lat=[float(i[2]),float(i[3])])        
        s1.extend(s2)
        sig_data = sig_data.append(pd.DataFrame(s1,['control_type','control_id','nodeid','masterid','distance','node_geom',
                                                    'control_geom','node_lon_lat','control_lon_lat']).T, ignore_index = True)
    except:
        f = [i[0],i[1]]
        print(f)
        f1=lon_lat_to_geom(cdb,[float(i[2]),float(i[3])])
        print(f1)
        f.extend([f1])
        print(f)
        failed_data=failed_data.append(pd.DataFrame(f,index = ['control_type','control_id','contol_geom']).T,ignore_index=True)

sig_data['node_lon_lat'] = map(lambda x: map(float,x.strip('POINT()').split(' ')), sig_data['node_lon_lat'])
sig_data['control_lon_lat'] = map(lambda x: map(float,x.strip('POINT()').split(' ')), sig_data['control_lon_lat'])
sig_data = sig_data.reindex(columns=['nodeid', 'masterid','control_type', 'control_id', 'node_lon_lat', 'control_lon_lat', 'distance'])
return [sig_data, failed_data]


(u'SIG', 10001, Decimal('-73.99247600'), Decimal('40.71507200'))
(u'SIG', 10002, Decimal('-74.00534300'), Decimal('40.72169400'))
(u'SIG', 10003, Decimal('-74.00521200'), Decimal('40.72055200'))
(u'SIG', 10004, Decimal('-74.00518800'), Decimal('40.71983800'))
(u'SIG', 10005, Decimal('-74.00517300'), Decimal('40.71900400'))
(u'SIG', 10006, Decimal('-73.99931100'), Decimal('40.71754900'))
(u'SIG', 10007, Decimal('-73.99683300'), Decimal('40.71509800'))
(u'SIG', 10008, Decimal('-73.99607900'), Decimal('40.71619600'))
(u'SIG', 10009, Decimal('-73.99748600'), Decimal('40.71400600'))
(u'SIG', 10010, Decimal('-74.01325000'), Decimal('40.70456300'))
(u'SIG', 10011, Decimal('-74.01154900'), Decimal('40.70495000'))
(u'SIG', 10013, Decimal('-74.00810200'), Decimal('40.71199300'))
(u'SIG', 10014, Decimal('-74.01426400'), Decimal('40.70444700'))
(u'SIG', 10015, Decimal('-74.00188300'), Decimal('40.71939600'))
(u'SIG', 10016, Decimal('-74.01084600'), Decimal('40.70879800'))
(u'SIG', 10017, Decimal('

(u'SIG', 10129, Decimal('-74.00050900'), Decimal('40.71098400'))
(u'SIG', 10130, Decimal('-74.01375600'), Decimal('40.70239800'))
(u'SIG', 10131, Decimal('-73.99269000'), Decimal('40.71437300'))
(u'SIG', 10132, Decimal('-73.99261000'), Decimal('40.71385300'))
(u'SIG', 10133, Decimal('-73.99238600'), Decimal('40.71318800'))
(u'SIG', 10134, Decimal('-73.99227900'), Decimal('40.71246000'))
(u'SIG', 10135, Decimal('-73.99019900'), Decimal('40.71405200'))
(u'SIG', 10136, Decimal('-73.98997200'), Decimal('40.71264700'))
(u'SIG', 10137, Decimal('-74.01637900'), Decimal('40.71008200'))
(u'SIG', 10138, Decimal('-74.00313800'), Decimal('40.70624900'))
(u'SIG', 10139, Decimal('-73.99181700'), Decimal('40.70981900'))
(u'SIG', 10140, Decimal('-74.00621200'), Decimal('40.70428600'))
(u'SIG', 10141, Decimal('-74.01254400'), Decimal('40.70148100'))
(u'SIG', 10142, Decimal('-73.99970100'), Decimal('40.71178000'))
(u'SIG', 10143, Decimal('-74.01439100'), Decimal('40.70359900'))
(u'SIG', 10144, Decimal('

(u'SIG', 10263, Decimal('-73.98915600'), Decimal('40.71670900'))
(u'SIG', 10264, Decimal('-73.98975500'), Decimal('40.71555300'))
(u'SIG', 10265, Decimal('-73.98754300'), Decimal('40.71983200'))
(u'SIG', 10266, Decimal('-73.98695100'), Decimal('40.72098200'))
(u'SIG', 10267, Decimal('-73.99261900'), Decimal('40.71885000'))
(u'SIG', 10268, Decimal('-73.99216100'), Decimal('40.71975000'))
(u'SIG', 10269, Decimal('-73.99309500'), Decimal('40.71792200'))
(u'SIG', 10270, Decimal('-73.99150400'), Decimal('40.72102600'))
(u'SIG', 10271, Decimal('-73.99091600'), Decimal('40.72218200'))
(u'SIG', 10272, Decimal('-73.98388000'), Decimal('40.71320300'))
(u'SIG', 10273, Decimal('-73.98165600'), Decimal('40.71445800'))
(u'SIG', 10274, Decimal('-73.99223500'), Decimal('40.71764500'))
(u'SIG', 10275, Decimal('-74.00140300'), Decimal('40.72255600'))
(u'SIG', 10276, Decimal('-74.00220800'), Decimal('40.72162200'))
(u'SIG', 10277, Decimal('-74.00036800'), Decimal('40.72379600'))
(u'SIG', 10278, Decimal('

(u'SIG', 10390, Decimal('-73.99050700'), Decimal('40.73056400'))
(u'SIG', 10391, Decimal('-73.99039200'), Decimal('40.73128900'))
(u'SIG', 10392, Decimal('-73.99021100'), Decimal('40.73199700'))
(u'SIG', 10393, Decimal('-73.99002700'), Decimal('40.73275700'))
(u'SIG', 10395, Decimal('-73.99635200'), Decimal('40.73225500'))
(u'SIG', 10396, Decimal('-73.99591500'), Decimal('40.73284800'))
(u'SIG', 10397, Decimal('-73.99549000'), Decimal('40.73343600'))
(u'SIG', 10398, Decimal('-73.99504900'), Decimal('40.73403400'))
(u'SIG', 10399, Decimal('-73.99458400'), Decimal('40.73467300'))
(u'SIG', 10400, Decimal('-73.99697300'), Decimal('40.73141000'))
(u'SIG', 10401, Decimal('-74.00064600'), Decimal('40.73722900'))
(u'SIG', 10402, Decimal('-74.00446000'), Decimal('40.73071800'))
(u'SIG', 10403, Decimal('-74.00355400'), Decimal('40.73222300'))
(u'SIG', 10404, Decimal('-74.00197000'), Decimal('40.73506900'))
(u'SIG', 10405, Decimal('-74.00285800'), Decimal('40.73360600'))
(u'SIG', 10407, Decimal('

(u'SIG', 10519, Decimal('-73.99363900'), Decimal('40.72729000'))
(u'SIG', 10520, Decimal('-73.99895400'), Decimal('40.72806700'))
(u'SIG', 10521, Decimal('-73.99735900'), Decimal('40.72994200'))
(u'SIG', 10522, Decimal('-73.99788600'), Decimal('40.72931000'))
(u'SIG', 10523, Decimal('-73.99991700'), Decimal('40.72693300'))
(u'SIG', 10524, Decimal('-74.00119100'), Decimal('40.72916800'))
(u'SIG', 10525, Decimal('-73.99861100'), Decimal('40.73221400'))
(u'SIG', 10526, Decimal('-74.00012300'), Decimal('40.73043000'))
(u'SIG', 10527, Decimal('-73.99960300'), Decimal('40.73104200'))
(u'SIG', 10528, Decimal('-73.99665900'), Decimal('40.72693100'))
(u'SIG', 10529, Decimal('-73.99936500'), Decimal('40.73006000'))
(u'SIG', 10530, Decimal('-73.99860400'), Decimal('40.72968600'))
(u'SIG', 10531, Decimal('-73.99479000'), Decimal('40.73159700'))
(u'SIG', 10532, Decimal('-73.99429900'), Decimal('40.73216500'))
(u'SIG', 10533, Decimal('-73.99381600'), Decimal('40.73273000'))
(u'SIG', 10534, Decimal('

...

(u'SIG', 50217, Decimal('-74.16237900'), Decimal('40.60461800'))
(u'SIG', 50218, Decimal('-74.16983400'), Decimal('40.57088000'))
(u'SIG', 50219, Decimal('-74.16991800'), Decimal('40.57397300'))
(u'SIG', 50220, Decimal('-74.16578700'), Decimal('40.59070800'))
(u'SIG', 50221, Decimal('-74.16947100'), Decimal('40.57937300'))
(u'SIG', 50222, Decimal('-74.16761600'), Decimal('40.58892700'))
(u'SIG', 50223, Decimal('-74.16868400'), Decimal('40.58591400'))
(u'SIG', 50224, Decimal('-74.16761600'), Decimal('40.58892700'))
(u'SIG', 50225, Decimal('-74.13149200'), Decimal('40.64039500'))
(u'SIG', 50226, Decimal('-74.16189900'), Decimal('40.59529800'))
(u'SIG', 50227, Decimal('-74.16893700'), Decimal('40.58221500'))
(u'SIG', 50228, Decimal('-74.16329400'), Decimal('40.60168300'))
(u'SIG', 50229, Decimal('-74.16102400'), Decimal('40.53989800'))
(u'SIG', 50230, Decimal('-74.16973100'), Decimal('40.57696500'))
(u'SIG', 50231, Decimal('-74.16964500'), Decimal('40.57786300'))
(u'SIG', 50232, Decimal('

(u'SIG', 50349, Decimal('-74.14478300'), Decimal('40.63795300'))
(u'SIG', 50350, Decimal('-74.15955600'), Decimal('40.53833500'))
(u'SIG', 50351, Decimal('-74.13069800'), Decimal('40.59859800'))
(u'SIG', 50352, Decimal('-74.08360700'), Decimal('40.59791200'))
(u'SIG', 50353, Decimal('-74.15122100'), Decimal('40.55331100'))
(u'SIG', 50354, Decimal('-74.14186200'), Decimal('40.59608300'))
(u'SIG', 50355, Decimal('-74.12930400'), Decimal('40.57467600'))
(u'SIG', 50356, Decimal('-74.07165700'), Decimal('40.60181600'))
(u'SIG', 50357, Decimal('-74.13231200'), Decimal('40.61025600'))
(u'SIG', 50358, Decimal('-74.11064400'), Decimal('40.59325000'))
(u'SIG', 50359, Decimal('-74.19015800'), Decimal('40.55157000'))
(u'SIG', 50360, Decimal('-74.16729300'), Decimal('40.52687700'))
(u'SIG', 50361, Decimal('-74.15852900'), Decimal('40.53057100'))
(u'SIG', 50362, Decimal('-74.20700600'), Decimal('40.54176300'))
(u'SIG', 50363, Decimal('-74.20798200'), Decimal('40.54170800'))
(u'SIG', 50364, Decimal('

(u'SIG', 50491, Decimal('-74.08783500'), Decimal('40.61531600'))
(u'SIG', 50494, Decimal('-74.12537000'), Decimal('40.62823200'))
(u'SIG', 50500, Decimal('-74.06183000'), Decimal('40.59630600'))
(u'SIG', 50505, Decimal('-74.10300700'), Decimal('40.60306700'))
(u'SIG', 50506, Decimal('-74.09239800'), Decimal('40.60707300'))
(u'SIG', 50511, Decimal('-74.12138800'), Decimal('40.57209100'))
(u'SIG', 50512, Decimal('-74.14589500'), Decimal('40.55136100'))
(u'SIG', 50516, Decimal('-74.17579000'), Decimal('40.62593800'))
(u'SIG', 50517, Decimal('-74.17242600'), Decimal('40.62748000'))
(u'SIG', 50518, Decimal('-74.16383100'), Decimal('40.57418500'))
(u'SIG', 50519, Decimal('-74.12847100'), Decimal('40.63367100'))
(u'SIG', 50523, Decimal('-74.14398200'), Decimal('40.63060100'))
(u'SIG', 50524, Decimal('-74.07020500'), Decimal('40.58657000'))
(u'SIG', 50525, Decimal('-74.11482000'), Decimal('40.57917700'))
(u'SIG', 50526, Decimal('-74.14149100'), Decimal('40.63348800'))
(u'SIG', 50527, Decimal('

(u'SIG', 50668, Decimal('-74.10931900'), Decimal('40.63006100'))
(u'SIG', 50669, Decimal('-74.13770300'), Decimal('40.60087400'))
(u'SIG', 50670, Decimal('-74.13446700'), Decimal('40.60111900'))
(u'SIG', 50671, Decimal('-74.09405300'), Decimal('40.61777500'))
(u'SIG', 50672, Decimal('-74.15592400'), Decimal('40.58509600'))
(u'SIG', 50673, Decimal('-74.16977500'), Decimal('40.54138500'))
(u'SIG', 50674, Decimal('-74.10766900'), Decimal('40.64527300'))
(u'SIG', 50675, Decimal('-74.13981900'), Decimal('40.61631100'))
(u'SIG', 50676, Decimal('-74.24180100'), Decimal('40.51635900'))
(u'SIG', 50678, Decimal('-74.07446800'), Decimal('40.62873400'))
(u'SIG', 50680, Decimal('-74.11968800'), Decimal('40.62618100'))
(u'SIG', 50681, Decimal('-74.15260200'), Decimal('40.61756100'))
(u'SIG', 50682, Decimal('-74.15155400'), Decimal('40.61751600'))
(u'SIG', 50683, Decimal('-74.15800700'), Decimal('40.59412400'))
(u'SIG', 50684, Decimal('-74.14026800'), Decimal('40.63625300'))
(u'SIG', 50685, Decimal('

SyntaxError: 'return' outside function (<ipython-input-12-c53e69f2818c>, line 21)

In [18]:
result.data[0][2]

Decimal('-73.99247600')

In [29]:
list([float(result.data[0][2]),float(result.data[0][3])])

[-73.992476, 40.715072]

In [34]:
lon_lat_to_geom(cdb,list([float(result.data[0][2]),float(result.data[0][3])]))

([-73.992476, 40.715072],)


'0101000020D7080000D5011E94BF192E41347ACDE4A4630841'